# Recommender System on Glocery Business

### Import Libraries

In [147]:
# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as bs
import requests # library to handle requests
import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries are imported.')

#import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library


Libraries are imported.


### Collecting Data With Beautiful Soup

In [43]:
#### beautifulsoup core: find tags with parent, sibling.

In [44]:
# getting data from internet
JiNanLink='http://tools.2345.com/yb.htm'
raw_JiNan_page= requests.get(JiNanLink).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_JiNan_page,'html',from_encoding='utf8')
#print(soup.prettify())

### Import Geospatial_Coordinates

In [45]:
# extracting the raw table inside that webpage
table = soup.find_all('table')

Postcode      = []
Borough       = []
Neighbourhood = []
table

# find shandong
for table_i in table:
    a=table_i.tr
    #print(a.next_sibling.next_sibling)

#table[24]
table_sd=table[24]
Borough       = []
Postcode      = []

# extracting a clean form of the table
for tr_cell in table_sd.find_all('tr'):
    counter = 1   
    for td_cell in tr_cell.find_all('td'):
        if counter == 1 : 
            Borough_var = str(td_cell.text)
            Borough.append(Borough_var)
        if counter == 3 : 
            Postcode_var = td_cell.text
            Postcode.append(Postcode_var)
        if counter == 4 and td_cell.text != " ":
            Borough_var = str(td_cell.text)
            Borough.append(Borough_var)
        if counter == 6 and td_cell.text != " ":
            Postcode_var = td_cell.text
            Postcode.append(Postcode_var)
        #append values
        counter +=1

# learn about copy and deepcopy
# pre: remove none type
none_type1='\xa0'
none_type2='邮政编码'
none_type3='市、县、区名'
while none_type1 in Borough:
    Borough.remove(none_type1)  
while none_type1 in Postcode:
    Postcode.remove(none_type1)
while none_type2 in Postcode:
    Postcode.remove(none_type2) 
while none_type3 in Borough:
    Borough.remove(none_type3) 

    
# then integrate with common values
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
for postcode_unique_element in unique_p:
    p_var = ''; b_var = '';  
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            Postcode_u.append(p_var)
            Borough_u.append(b_var)
#Postcode_u
#Borough_u
#table_sd

num of unique Postal codes: 123


In [72]:
#pd.to_csv('df_sd.csv')
df_sd = pd.read_csv('df_sd.csv')

In [74]:
# make dic and df
sd_dict = {'Postcode':Postcode_u, 'Borough':Borough_u}
df_sd = pd.DataFrame.from_dict(sd_dict)
df_sd.drop(0,axis=0,inplace=True)
#df_sd.set_index('Postcode', inplace=True)
jn_list=['市中区','历下区','天桥区','槐荫区','历城区','长清区','章丘市','济阳县','钢城区','平阴县','商河县']
df_jn=df_sd[df_sd['Borough'].isin(jn_list)]
#delete wrong records
df_jn.drop([38,156,147],inplace=True)

# from website
#loc_dic={'Latitude':[37.32,36.98,36.72,36.55,36.68,36.28,36.67,35.40,36.65,36.68],'longtitude':[117.15,117.22,117.53,116.73,117.07,116.45,117.08,116.58,116.93,116.98]}
df_jn['Latitude']=[37.32,36.55,36.72,36.98,36.68,36.67,35.40,36.65,36.68,36.28]
df_jn['Longitude']=[117.15,116.73,117.53,117.22,117.07,117.08, 116.58,116.93,116.98,116.45]
#pd.to_csv('df_jn.csv')
df_jn.to_csv('df_jn.csv')
df_jn

/Users/aestheticism/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/aestheticism/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,Borough,Latitude,Longitude
1,251600,商河县,37.32,117.15
20,250300,长清区,36.55,116.73
36,250200,章丘市,36.72,117.53
62,251400,济阳县,36.98,117.22
95,250100,历城区,36.68,117.07
115,250000,历下区,36.67,117.08
116,250000,市中区,35.40,116.58
117,250000,槐荫区,36.65,116.93
118,250000,天桥区,36.68,116.98
149,250400,平阴县,36.28,116.45


### Map Jinan

In [76]:
df_jn=pd.read_csv('df_jn.csv')
df_jn

,Unnamed: 0,Postcode,Borough,Latitude,Longitude
0,1,251600,商河县,37.32,117.15
1,20,250300,长清区,36.55,116.73
2,36,250200,章丘市,36.72,117.53
3,62,251400,济阳县,36.98,117.22
4,95,250100,历城区,36.68,117.07
5,115,250000,历下区,36.67,117.08
6,116,250000,市中区,35.40,116.58
7,117,250000,槐荫区,36.65,116.93
8,118,250000,天桥区,36.68,116.98
9,149,250400,平阴县,36.28,116.45


In [140]:
df_jn=pd.read_csv('df_jn.csv')
# for the city Jinan, latitude and longtitude are manually extracted via baidu search
Jinan_latitude = 36.67 ; Jinan_longitude = 116.98
map_Jinan = folium.Map(location = [Jinan_latitude, Jinan_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough in zip(df_jn['Latitude'], df_jn['Longitude'], df_jn['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Jinan)  
#map_Jinan

### Define Foursquare Credentials and Version

In [141]:
CLIENT_ID = 'DBMBAOKP15B3BFT0T4O4KODS4HU2IGKTUE0OESSHM0Q15TC1' # your Foursquare ID
CLIENT_SECRET = 'QGZTNMNUIAIOFSPHLM0EJ5G4WTVKLSMB2WGNT54ORI5H0LIR' # your Foursquare Secret
VERSION = '20200210' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DBMBAOKP15B3BFT0T4O4KODS4HU2IGKTUE0OESSHM0Q15TC1
CLIENT_SECRET:QGZTNMNUIAIOFSPHLM0EJ5G4WTVKLSMB2WGNT54ORI5H0LIR


###  Top 100 venues that are in 'Lixia', from Jinan.

In [142]:
neighborhood_latitude = df_jn.loc[5, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_jn.loc[5, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_jn.loc[5, 'Borough'] # neighborhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "历下区" are 36.67, 117.08.


In [143]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# display URL
# url

In [144]:
results = requests.get(url).json()
#results

In [148]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,山东省博物馆新馆,Museum,36.656844,117.089116
1,Lydia (莉迪亚意式餐厅),Italian Restaurant,36.679346,117.049908
2,KFC (肯德基),Fast Food Restaurant,36.675263,117.050926
3,McDonald's (麦当劳),Fast Food Restaurant,36.683529,117.057588
4,银座商城华信店,Shopping Mall,36.687907,117.077113


### Explore Nearby Venues 

In [178]:
# get nearby venues

LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [179]:
Jinan_venues = getNearbyVenues(names=df_jn['Borough'],
                                   latitudes=df_jn['Latitude'],
                                   longitudes=df_jn['Longitude']
                                  )

商河县
长清区
章丘市
济阳县
历城区
历下区
市中区
槐荫区
天桥区
平阴县


In [180]:
df_jn
nearby_venues

,name,categories,lat,lng
0,山东省博物馆新馆,Museum,36.656844,117.089116
1,Lydia (莉迪亚意式餐厅),Italian Restaurant,36.679346,117.049908
2,KFC (肯德基),Fast Food Restaurant,36.675263,117.050926
3,McDonald's (麦当劳),Fast Food Restaurant,36.683529,117.057588
4,银座商城华信店,Shopping Mall,36.687907,117.077113
5,RT-mart (大润发),Grocery Store,36.684578,117.058264


In [183]:
# venue: separated place
print(Jinan_venues .shape)
print('There are {} uniques categories.'.format(len(Jinan_venues['Venue Category'].unique())))
Jinan_venues.groupby('Neighborhood').count()

(12, 7)
There are 8 uniques categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
历下区,3,3,3,3,3,3
历城区,2,2,2,2,2,2
天桥区,4,4,4,4,4,4
市中区,1,1,1,1,1,1
平阴县,1,1,1,1,1,1
章丘市,1,1,1,1,1,1


### One Hot Encoding and Most Common Venue

In [184]:
# one hot encoding
Jinan_onehot = pd.get_dummies(Jinan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Jinan_onehot ['Neighborhood'] = Jinan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Jinan_onehot.columns[-1]] + list(Jinan_onehot.columns[:-1])
Jinan_onehot = Jinan_onehot[fixed_columns]

Jinan_onehot.head(12)
Jinan_onehot.shape

(12, 9)

In [124]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [126]:
Jinan_venues_grouped =Jinan_onehot.groupby('Neighborhood').mean().reset_index()

#give a num
num = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Jinan_venues_grouped['Neighborhood']

for ind in np.arange(Jinan_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Jinan_venues_grouped.iloc[ind, :], num)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,历下区,Chinese Restaurant,Convenience Store,Park,Ice Cream Shop,Hotel,Hostel,Fast Food Restaurant,Arts & Crafts Store
1,历城区,Hostel,Fast Food Restaurant,Park,Ice Cream Shop,Hotel,Convenience Store,Chinese Restaurant,Arts & Crafts Store
2,天桥区,Hotel,Fast Food Restaurant,Arts & Crafts Store,Park,Ice Cream Shop,Hostel,Convenience Store,Chinese Restaurant
3,市中区,Ice Cream Shop,Park,Hotel,Hostel,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Arts & Crafts Store
4,平阴县,Hotel,Park,Ice Cream Shop,Hostel,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Arts & Crafts Store
5,章丘市,Park,Ice Cream Shop,Hotel,Hostel,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Arts & Crafts Store


### Cluster Groups

In [186]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

#Jinan_data = Jinan_data.drop(13)
# set number of clusters
kclusters = 5

Jinan_grouped_clustering = Jinan_venues_grouped.drop('Neighborhood', axis=1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Jinan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16

array([4, 1, 2, 0, 2, 3], dtype=int32)

In [194]:
# choose grouped neighborhood
jn_list2=['历下区','历城区','天桥区','市中区','平阴县','章丘市']
df_jn2=df_jn[df_jn['Borough'].isin(jn_list2)]
df_jn2.columns=['Unnamed: 0','Postcode','Neighborhood','Latitude','Longitude']
df_jn2.drop(['Unnamed: 0'],axis=1,inplace=True)
Jinan_merged = df_jn2

# add clustering labels
Jinan_merged['Cluster Labels'] = kmeans.labels_

# merge two datasets to add latitude/longitude for each neighborhood
Jinan_merged = Jinan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Jinan_merged

/Users/aestheticism/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Postcode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
2,250200,章丘市,36.72,117.53,4,Park,Ice Cream Shop,Hotel,Hostel,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Arts & Crafts Store
4,250100,历城区,36.68,117.07,1,Hostel,Fast Food Restaurant,Park,Ice Cream Shop,Hotel,Convenience Store,Chinese Restaurant,Arts & Crafts Store
5,250000,历下区,36.67,117.08,2,Chinese Restaurant,Convenience Store,Park,Ice Cream Shop,Hotel,Hostel,Fast Food Restaurant,Arts & Crafts Store
6,250000,市中区,35.40,116.58,0,Ice Cream Shop,Park,Hotel,Hostel,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Arts & Crafts Store
8,250000,天桥区,36.68,116.98,2,Hotel,Fast Food Restaurant,Arts & Crafts Store,Park,Ice Cream Shop,Hostel,Convenience Store,Chinese Restaurant
9,250400,平阴县,36.28,116.45,3,Hotel,Park,Ice Cream Shop,Hostel,Fast Food Restaurant,Convenience Store,Chinese Restaurant,Arts & Crafts Store


In [199]:
address_Jinan= 'Jinan'
latitude_Jinan = 43.773077
longitude_Jinan = -79.257774
print('The geograpical coordinate of Jinan are {}, {}.'.format(latitude_Jinan, longitude_Jinan))

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


map_scarborough= folium.Map(location=[latitude_Jinan, longitude_Jinan], zoom_start=12)

# create map
map_clusters = folium.Map(location = [latitude_Jinan, longitude_Jinan], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Jinan_merged['Latitude'], Jinan_merged['Longitude'], Jinan_merged['Neighborhood'], Jinan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_Jinan)  
    
map_Jinan

The geograpical coordinate of Jinan are 43.773077, -79.257774.


### Center of Clusters

In [215]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
#Jinan_onehot.drop(['Neighborhood'],inplace=True,axis=1)
means_df.columns = Jinan_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Arts & Crafts Store,Chinese Restaurant,Convenience Store,Fast Food Restaurant,Hostel,Hotel,Ice Cream Shop,Park,Total Sum
G1,0.000,0.000000,0.000000,0.000,0.0,0.00,1.0,0.0,1.0
G2,0.000,0.000000,0.000000,0.500,0.5,0.00,0.0,0.0,1.0
G3,0.125,0.000000,0.000000,0.125,0.0,0.75,0.0,0.0,1.0
G4,0.000,0.000000,0.000000,0.000,0.0,0.00,0.0,1.0,1.0
G5,0.000,0.666667,0.333333,0.000,0.0,0.00,0.0,0.0,1.0


#### Finally, we can get to the conclusion that G1 proves to be the best.